<a href="https://colab.research.google.com/github/vkjadon/ros2/blob/master/ultrasonic_publisher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import rclpy
from rclpy.node import Node
from std_msgs.msg import Float32
import RPi.GPIO as GPIO
import time

class UltrasonicPublisher(Node):
    def __init__(self):
        super().__init__('ultrasonic_publisher')
        self.publisher_ = self.create_publisher(Float32, 'ultrasonic_distance', 10)
        self.timer = self.create_timer(1.0, self.timer_callback)

        # GPIO Mode (BOARD / BCM)
        GPIO.setmode(GPIO.BCM)

        # Set GPIO Pins
        self.GPIO_TRIGGER = 18
        self.GPIO_ECHO = 24

        # Set GPIO direction (IN / OUT)
        GPIO.setup(self.GPIO_TRIGGER, GPIO.OUT)
        GPIO.setup(self.GPIO_ECHO, GPIO.IN)

    def timer_callback(self):
        distance = self.read_distance()
        self.publisher_.publish(Float32(data=distance))
        self.get_logger().info(f'Publishing: {distance} cm')

    def read_distance(self):
        # Set Trigger to HIGH
        GPIO.output(self.GPIO_TRIGGER, True)

        # Set Trigger after 0.01ms to LOW
        time.sleep(0.00001)
        GPIO.output(self.GPIO_TRIGGER, False)

        start_time = time.time()
        stop_time = time.time()

        # Save StartTime
        while GPIO.input(self.GPIO_ECHO) == 0:
            start_time = time.time()

        # Save time of arrival
        while GPIO.input(self.GPIO_ECHO) == 1:
            stop_time = time.time()

        # Time difference between start and arrival
        time_elapsed = stop_time - start_time
        # Multiply with the sonic speed (34300 cm/s) and divide by 2
        distance = (time_elapsed * 34300) / 2

        return distance

    def __del__(self):
        GPIO.cleanup()

def main(args=None):
    rclpy.init(args=args)
    node = UltrasonicPublisher()
    try:
        rclpy.spin(node)
    except KeyboardInterrupt:
        pass
    finally:
        node.destroy_node()
        rclpy.shutdown()

if __name__ == '__main__':
    main()
